# Imports

In [ ]:
import re
from unicodedata import normalize
import html
from tqdm.notebook import tqdm
import pickle
import gc

from datetime import date
import dateparser

import Levenshtein
from difflib import SequenceMatcher

# Load articles

In [ ]:
data_persons = pickle.load(open("../Data/data_v2_persons.p", "rb"))

____

In [ ]:
def get_attr_frequency(data: list):
    
    '''This function returns the keys sorted by frecuency of appearence in the dataset'''
    
    attr_freq = {}
    
    for a in data:
        for k in list(a.keys()):

            if k in attr_freq.keys():
                attr_freq[k] += 1
            else:
                attr_freq[k] = 1

    attr_freq = {k: v for k, v in sorted(attr_freq.items(), key=lambda item: item[1], reverse=True)}

    num_attrs = len(attr_freq.keys())
    
    print(f'# of distinct attributes: {num_attrs}')
    
    
    return attr_freq

____

## Remove meaningless keys from attributes

In [ ]:
def remove_meaningless_keys(data: list, keys_to_remove):
    
    '''This function removes meaningless keys from the dataset''' 
    
    for a in data:
        attrs = list(a.keys())
        for attr in attrs:
            for key in keys_to_remove:
                if key==attr:
                    del a[attr]
            
    return data

####

to_remove = ['imagen','alt','identificador youtube de video','iamgen','bandcamp','cwidth',
 'oleft','image width','imagen https://i.pinimg.com/originals/67/50/d4/6750d4c26f054b8dde911767a35673d4.jpg',
 'pie de imagen','pagina web','tamaño de imagen','firma','imdb','tamaño','facebook',
 'twitter','tamaño imagen','instagram','color de ojos','color de cabello','pie de foto',
 'img capt','pieimagen','foto','img size','myspace','pie imagen','pie de pagina',
 '280px','numero telefonico','image','caption','web','url2','patrimonio','youtube',
 'tamañoimagen','sitio oficial','img','twitter oficial','url1','pie imagenes','mail',
 'sitio  web','redes sociales','autografo','sitio web','url','web oficial','sitio web3',
 'sitio web oficial','pieimagrn','sitio webarchivo','facebok','urrl','pagina web oficial',
 'imbd','image size','tamaño de foto','sitioweb','wikidata','textoimagen','grupo sanguineo',
 'color','ancho','escudo','tamano','bsize','tam de imagen','image tamaño','tipo de sangre',
 'soundcloud','subtitulo','website','peso','pie','equipo de futbol','pagina web3',
 'numero de peliculas','sitio externo','subtitulo\ne','prenda de vestir','subtitutlo',
 'peliculas','titulo de imagen','deporte','id','belleza','ubtitulo','descripcion',
 'logo','salario','ranking [[dj mag]]','landscape','medidas','snapchat','bhrnryngb murmullo',
 'archivo','epigrafe','bautizo','presidencia de honor','posicion filosofica','club actual',
 'equipo','fans','pais preferido','paisaje','gerente','cine','signo zodiacal',
 'zodiaco chino','labels','modulo','medio de comunicacion','programas','partido politico',
 'programa','partido','formato','medio','final','escudo2','vine','filme','television',
 '1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','cabecera',
 'inicio','fecha','etiqueta','immagine bandiera nazione','notas','cacahuate',
 'nombre  anuel aa','fandoom','libros relacionados','relacionados| [[ismael quintana]]',
 'nombre  tambien muy amada por sus fans','shows','presentaciones asociadas','editorial',
 'fecha de nacimiento igual a la de la guaren.alemam','enlistamiento','associated acts',
 'ann aop','etnia','canal','obras','afiliaciones','serie','sede','empresa','carrera',
 'distribuidora','ann m','tipo de','fecha de operacion','modelos','nac','inicio2',
 'ultimo concierto','actos relacionados','filiacoes','fondo','cargo','cargo2',
 'background','location','sitio','distrito','cancion destacada','trabajos notables',
 'obras destacadas','obrasdestacadas','\'\'\'discografica','hijo(a)s','parientes',
 'conocido','conocido por', 'estado civil', 'familia', 'cantidad de hermanos', 
 'comprometida', 'casada', 'esatado civil', 'nombre en japones','rango en vivo',
 'rango en estudio', 'rango', 'rango vocal', 'formacion', 'enterrado','lugar de descanso',
 'lugar de entierro', 'cargos criminales', 'condena', 'situacion penal',
 'estatura', 'altura', 'altezza', 'estatura:', 'date', 'author', 'obras notables',
 'publisher', 'work', 'subtítulo', 'rol debut', 'página web', 'posicion', 
 'associated_acts', 'accessdate', 'contratacion', 'año retiro',
 'numero total de albumes publicados', 'fechaarchivo', 'estado',
 ', [[los morochucos]],  [[los kipus]], [[artistas varios]]url', 'afiliacion',
 'nombre               anuel aa', 'numero albumes de estudio', 'familiares',
 'causa/razon', 'upright', 'asociados a', 'asociado a', 'musicos acompañantes',
 'ex miembros', 'thumb|pablo canela, insigne violinista caroreño|tamaño',
 'predecesora', 'relacion', 'nombre     tambien muy amada por sus fans',
 'antiguos miembros', '(estatura)', 'fechaacceso', 'iempo', 'causa',
 'numero total publico albumes', 'album de estudio numero', 'estudiantes',
 'compilacion de musica', 'album publicados', 'estudio', 'en vivo', 'colecciones',
 'gustos musicales', 'título', 'miembros iniciales', 'ficha', 'obra', 'years_active',
 'educacion', 'empleador', 'otros miembros', 'miembros', 'past members', 'area',
 'asociados', 'mide', 'agencia', 'productor', 'agencia artistica', 'representante',    
 'discogs', 'nazione gruppo', 'movimiento', 'original', 'titulo', 'compañeros',
 'tipo', 'nuevos miembros', 'miembros anteriores a su agrupacion',
 'current members', 'miembro',  '[[religion]]', 'nacido / a',
 'apellidos', 'apellido','nietos', 'nieto favorito', 'abuelo',
 'sobrinos', 'bisnieto',]

data_persons = remove_meaningless_keys(data_persons, to_remove)

####


print(f'# of articles about persons: {len(data_persons)}')
attr_frequency_persons = get_attr_frequency(data_persons)

## Collapse and clean attributes

In [ ]:
meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio',
         'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']

In [ ]:
def extract_date(text: str):
    
    ''' ... '''
        
    pat_st = '('
    pat_01 = '\d*\s*[de]*\s*enero\s*[de]*l*\s*\d+|'
    pat_02 = '\d*\s*[de]*\s*febrero\s*[de]*l*\s*\d+|'
    pat_03 = '\d*\s*[de]*\s*marzo\s*[de]*l*\s*\d+|'
    pat_04 = '\d*\s*[de]*\s*abril\s*[de]*l*\s*\d+|'
    pat_05 = '\d*\s*[de]*\s*mayo\s*[de]*l*\s*\d+|'
    pat_06 = '\d*\s*[de]*\s*junio\s*[de]*l*\s*\d+|'
    pat_07 = '\d*\s*[de]*\s*julio\s*[de]*l*\s*\d+|'
    pat_08 = '\d*\s*[de]*\s*agosto\s*[de]*l*\s*\d+|'
    pat_09 = '\d*\s*[de]*\s*septiembre\s*[de]*l*\s*\d+|'
    pat_10 = '\d*\s*[de]*\s*octubre\s*[de]*l*\s*\d+|'
    pat_11 = '\d*\s*[de]*\s*noviembre\s*[de]*l*\s*\d+|'
    pat_12 = '\d*\s*[de]*\s*diciembre\s*[de]*l*\s*\d+'
    pat_en = ')'
    
    pattern = (pat_st + pat_01 + pat_02 + pat_03 + pat_04 + pat_05 + pat_06 + 
               pat_07 + pat_08 + pat_09 + pat_10 + pat_11 + pat_12 + pat_en)

    dates = re.findall(pattern, text)
    
    if len(dates)==0:
        return text, ''
    
    elif len(dates)==1:
        text = ' '.join(text.split(dates[0]))
        date = dates[0]
        return text, date
    
    else:
        print('*!*!*!*!*!*')
        print(text)
        
        return '',''

    
####


def extract_rango_fechas(text: str):
    
    ''' ... '''
    
    pattern = r'\d{4}\s*?[\−\-\–]\s*?\d{4}\b'
    group = re.findall(pattern, text)
    if len(group)==1:
        rango_fecha = group[0]
        año_1 = rango_fecha[:4]
        año_2 = rango_fecha[-4:]
        return año_1, año_2
    elif len(group)==0:
        pass
    else:
        print('+ de un rango de fechas...')
        return '',''
        
    pattern = r'\d{4}\s*?[\−\-\–]\s*?\d{2}'
    group = re.findall(pattern, text)
    if len(group)==1:
        rango_fecha = group[0]
        año_1 = rango_fecha[:4]
        año_2 = año_1[:2] + rango_fecha[-2:]
        return año_1, año_2
    elif len(group)==0:
        pass
    else:
        print('+ de un rango de fechas...')
        return '',''
    
    pattern = r'\d{4}\s*?[\−\-\–]\s*?\S.*?\b'
    group = re.findall(pattern, text)
    if len(group)==1:
        rango_fecha = group[0]
        año_1 = rango_fecha[:4]
        año_2 = 'actualidad'
        return año_1, año_2
    elif len(group)==0:
        pass
    else:
        print('+ de un rango de fechas...')
        return '',''

    pattern = r'\d{4}\b'
    group = re.findall(pattern, text)
    if len(group)==1:
        año_1 = group[0]
        año_2 = ''
        return año_1, año_2
    elif len(group)==0:
        pass
    else:
        print('+ de un rango de fechas...')
        return '',''
    
    return '',''

####


def remove_rango_fechas(text: str):
    
    ''' ... '''
    
    pattern = r'\d{4}\s*?[\−\-\–]\s*?\d{4}\b'
    group = re.findall(pattern, text)
    for g in group:
        text = text.replace(g,' ')

        
    pattern = r'\d{4}\s*?[\−\-\–]\s*?\d{2}'
    group = re.findall(pattern, text)
    for g in group:
        text = text.replace(g,' ')
    
    pattern = r'\d{4}\s*?[\−\-\–]\s*?\S.*?\b'
    group = re.findall(pattern, text)
    for g in group:
        text = text.replace(g,' ')

    pattern = r'\d{4}\b'
    group = re.findall(pattern, text)
    for g in group:
        text = text.replace(g,' ')
    
    text = clean_text_I(text)
    text = clean_text_II(text)
    
    return text

####


def decode_fecha(text: str):
    
    ''' ... '''
    
    pattern = '=<=FECHA=<=.*?=>=FECHA=>='
    elements = re.findall(pattern, text)

    if len(elements)==0:
        
        return text, ''
    
    elif len(elements)==1:
        
        text = ' '.join(text.split(elements[0])).strip()
        fecha = elements[0][11:-11].strip()
        
        f = fecha.split('/')
        
        pattern = '\d+\/\d+\/\d+'
        dates = re.findall(pattern, fecha)
        
        if len(dates)>0:
            
                try:
                    fecha = str(int(f[0])) + ' de ' + meses[int(f[1])-1] + ' de ' + f[2]
                except:
                    return text, ''
            
        else:
            
            pattern = '\d+\/\D+\/\d+'
            dates = re.findall(pattern, fecha)
            
            if len(dates)>0:
                try:
                    fecha = str(int(f[0])) + ' de ' + f[1] + ' de ' + f[2]
                except:
                    return text, ''
        
        return text, fecha
    
    else:
        
        text = ' '.join(text.split(elements[0])).strip()
        fecha = elements[0][11:-11].strip()
        
        f = fecha.split('/')
        
        pattern = '\d+\/\d+\/\d+'
        dates = re.findall(pattern, fecha)
        
        if len(dates)>0:
            fecha = str(int(f[0])) + ' de ' + meses[int(f[1])-1] + ' de ' + f[2]
            
        else:
            pattern = '\d+\/\D+\/\d+'
            dates = re.findall(pattern, fecha)
            
            if len(dates)>0:
                fecha = str(int(f[0])) + ' de ' + f[1] + ' de ' + f[2]
        
        return text, fecha

    
####


def decode_fechas(text: str):
    
    ''' ... '''
        
    pattern = '=<=FECHAS=<=.*=>=FECHAS=>='
    elements = re.findall(pattern, text)
    
    fecha_1 = ''
    fecha_2 = ''
    
    if len(elements)==0:
        
        return text, '', ''
    
    elif len(elements)==1:
        
        text = ' '.join(text.split(elements[0])).strip()
        
        fecha = elements[0][12:-12].strip()
        
        f1 = fecha.split('-')[0]
        f_1 = f1.split('/')
        pattern = '\d+\/\d+\/\d+'
        dates = re.findall(pattern, f1)
        
        if len(dates)>0:
            fecha_1 = str(int(f_1[0])) + ' de ' + meses[int(f_1[1])-1] + ' de ' + f_1[2]
            
        else:
            pattern = '\d+\/\D+\/\d+'
            dates = re.findall(pattern, f1)
            
            if len(dates)>0:
                fecha_1 = str(int(f_1[0])) + ' de ' + f_1[1] + ' de ' + f_1[2]

        
        f2 = fecha.split('-')[1]
        f_2 = f2.split('/')
        pattern = '\d+\/\d+\/\d+'
        dates = re.findall(pattern, f2)
        
        if len(dates)>0:
            fecha_2 = str(int(f_2[0])) + ' de ' + meses[int(f_2[1])-1] + ' de ' + f_2[2]
            
        else:
            pattern = '\d+\/\D+\/\d+'
            dates = re.findall(pattern, f2)
            
            if len(dates)>0:
                fecha_2 = str(int(f_2[0])) + ' de ' + f_2[1] + ' de ' + f_2[2]
                
        return text, fecha_1, fecha_2
    
    else:
        print('*!*!*!*!*')
        print(elements)
        
        # force_stop
        
        return text, '', ''
    
####

In [ ]:

def remove_age(text: str):
    
    
    pattern = r'\(*\s*\d*\s*\Waños*(\W|\b)\s*\)*'
    text = re.sub(pattern, '', text)
    
    return text


####


def get_born_death_dates(dates, b_d='d'):
    
    dates = [date for date in dates if date!='']
    dates = [dateparser.parse(date) for date in dates]
    dates = [date for date in dates if date!=None]
    dates = list(dict.fromkeys(dates))
    
    if len(dates)==0:
        return '', ''

    elif len(dates)==1:
        fecha = str(dates[0].day) + ' de ' + meses[dates[0].month-1] + ' de ' + str(dates[0].year)
        
        if b_d=='d':
            return '', fecha
        else:
            return fecha, ''

    else:
        nacimiento = str(sorted(dates)[0].day) + ' de ' + meses[sorted(dates)[0].month-1] + ' de ' + str(sorted(dates)[0].year)
        muerte = str(sorted(dates)[-1].day) + ' de ' + meses[sorted(dates)[-1].month-1] + ' de ' + str(sorted(dates)[-1].year)
        return nacimiento, muerte
    
####


def decode_reasons(text: str):
    
    pattern = '==DIVORCIO=='
    r = re.findall(pattern, text)
    if len(r)>0:
        text = text.replace(pattern, '')
        text = clean_text_I(text)
        text = clean_text_II(text)
        return text, 'divorcio'

    pattern = '==FALLECIMIENTO=='
    r = re.findall(pattern, text)
    if len(r)>0:
        text = text.replace(pattern, '')
        text = clean_text_I(text)
        text = clean_text_II(text)
        return text, 'fallecimiento'
    
    pattern = '==PAREJA=='
    r = re.findall(pattern, text)
    if len(r)>0:
        text = text.replace(pattern, '')
        text = clean_text_I(text)
        text = clean_text_II(text)
        return text, 'pareja'
    
    pattern = '†'
    r = re.findall(pattern, text)
    if len(r)>0:
        print('†††')
        text = text.replace(pattern, '')
        text = clean_text_I(text)
        text = clean_text_II(text)
        return text, 'fallecimiento'
    
    return text, ''
    
####


def clean_premios(text: str):

    to_clean_l = ['Lista de premios','Véase Premios', 'Véase Premios y nominaciones','• ']
    
    for e in to_clean_l:
        text = text.replace(e,'')
        
    return text
    
####


def remove_links(text: str):
    
    ''' ... '''
    
    pattern = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    links = re.findall(pattern, text)
    links = [link[0] for link in links] 
    
    for link in links:
        text = ''.join(text.split(link))
    
    return text

####


def replace_info_from_brackets(text: str):
    
    ''' ... '''
    
    brackets_pattern = '\[\[.*?\]\]'
    brackets = re.findall(brackets_pattern, text)
    
    for bracket in brackets:
        parts = bracket.split('%%')
        if len(parts)==1:
            replace_text = bracket.replace('[','').replace(']','')     
        else:
            replace_text = parts[1].replace(']','')
        
        text = text.replace(bracket, replace_text)

    return text

####


def clean_text_I(text: str):
    
    ''' ... '''

    
    separators = ['<br/>(', '<br />(', '<br/ >(', '<br>(', '<Br>(', '<br><small>(',
                  '<br /><small>(']
    for s in separators:
        text = text.replace(s, '(')
        
    separators = ['\n', 
                  '<br>', '<br >',
                  '<Br>', '<Br >',
                  '<br/>', '<br />', '<br/ >', '<br / >',
                  '<Br/>', '<Br />', '<Br/ >', '<Br / >',                  
                  '&nbsp;', '&nbsp',
                  '<li>',
                  '<span>', '</span>',
                  '<small>', '</small>', '}}',
                  "'''"]
    
    for s in separators:
        text = text.replace(s, ' ')
    
    
    text = re.sub(r'\d\dpx', '', text )

    return text

####


def clean_text_II(text: str, texto=False):
    
    ''' ... '''
    
    if text=='no':
        return ''
    
    if len(text)>0:
        text = text.strip()
        if len(text)>0:
            if text[0] == '*':
                text = text[1:]
        
    text = re.sub("\'\'\'", '', text)
    
    text = re.sub('\(\(+', '((', text)   
    text = re.sub('\)\)+', '))', text)
    
    text = re.sub('\(\s*?[\−\-\–,]\s*?\)','', text)  
    text = re.sub('\(+\s*\)+', '', text)
    
    text = re.sub('\(\s*?fallecido\s*?\)', ' ', text)
    text = re.sub('\(\s*?presente\s*?\)', ' ', text)
    text = re.sub('\s+\.', ' ', text)
    text = re.sub('\(\s*?-\s*?\)', ' ', text)
    text = re.sub('\(\s*?Desde\s*?\)', ' ', text)
    text = re.sub('\(\s*?desde\s*?\)', ' ', text)
    
    if texto:
        text = (text.replace('_',' ').replace('\n', ' ')
                    .replace('\'\'', ' ')
                    .replace('{', ' ').replace('}', ' ')
                    .replace('[', ' ').replace(']', ' ')
                    .replace(r'\(*','(').replace(r'\)*',')')
                    .replace(r'\(\s*\)',' ')
                    .replace('\xa0',' ')
                    .strip())
        
        
        text = re.sub('\s*\.', '.', text)
        text = re.sub('\s*,', ',', text)
        text = re.sub('\s*;', ';', text)
        text = re.sub('\.+', '.', text)
        text = re.sub(',+', ',', text)
        text = re.sub('\.,', '.', text)
        text = re.sub('\s*\)', ')', text)
        text = re.sub(',+\)', ')', text)
        text = re.sub('\(;','(', text)
        text = re.sub('\(\s*', '(', text)
        text = re.sub(',-', '-', text)
        text = re.sub('\(\(', '(', text)
        text = re.sub(';\)', ')', text)
        text = re.sub('\(,', ')', text)
        text = re.sub(',\.', '.', text)
        
        text = text.strip()
        
     
    else:  
        text = (text.replace('_',' ').replace('\n', ' ')
                    .replace('\'\'', ' ').replace('"', ' ')
                    .replace('{', ' ').replace('}', ' ')
                    .replace('[', ' ').replace(']', ' ')
                    .replace('«',' ').replace('»',' ')
                    .replace(r'\(*','(').replace(r'\)*',')')
                    .replace(r'\(\s*\)',' ')
                    .strip())
        
        try:
            if text[-1]=='.':
                text = text[:-1]
        except:
            pass

    
    #text = text.replace('(', ' ').replace(')', ' ').

    # -> NFD & eliminar diacríticos
    text = re.sub(r'([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+',
                      r'\1', text, 0, re.I)

    # normalize( "NFD", text)
    
    # -> NFC
    text = normalize('NFC', text)
    
    text = re.sub(' +', ' ', text)
    
    text = text[:1].upper() + text[1:]

    return text

####


def decode_references(text: str, references):

    ''' ... '''

    pattern = '%«%.*?%»%'
    try:
        tags = re.findall(pattern, text)
    except:
        tags = []
    
    for tag in tags:
        
        position = re.findall(r'\d*', tag)
        position = [int(p) for p in position if len(p)>0][0]
        
        t = references[position]

        seps = ['br/>', 'br', 'ref']
        if (t['tag'] in seps):
            text = text.replace(tag, '&&&&')
        else:
            text = text.replace(tag, t['value'])

    text = text.replace('&&&&(', ' (').replace('&&&& (', ' (')
    text = text.replace('&&&&', ',')
    
    return text

####


def decode_brackets(text: str, brackets):

    ''' ... '''
        
    pattern = '%{%.*?%}%'
    tags = re.findall(pattern, text)
    
    for tag in tags:
        
        position = re.findall(r'\d*', tag)
        position = [int(p) for p in position if len(p)>0][0]
        
        t = (brackets[position].replace('{{','').replace('}}','')
                              .replace('\n', ',').replace('<br>',' '))
        
        t_elements = t.split('|')
        
        to_remove = ['()=small', '()=smaller', 'small=']
        t_elements = [t for t in t_elements if t not in to_remove]
        
        header = t_elements[0].strip().lower()
        
        values_l = ['small', ]
        
        matrimonio_l = ['matrimonio', 'marriage', 'natrimonio', ]
        
        fecha_l = ['fecha', 'edad', 'start date', 'start date', 'fecha de inicio',
                   'fecha de muerte y edad', 'fecha de muerte', 'fecha de nacimiento',
                   'fecha de muerte', 'muerte', 'fecha de termino', 'fecha2',
                   'julgregfecha', 'death date and age', 'death date',
                   'birth date and age', 'birth date', 'fecha de nacimiento y edad']
        
        flatlist_l = ['flatlist', 'lista horizontal', 'desplegable', 'hlist',
                      'lista plana', 'lista desplegable', 'lista', 'lista simple', 
                      'flat list', 'ubl', 'lsv', 'unbulleted list', 'plainlist']
        
        descartar_l = ['cita web', 'citaweb', 'enlace roto', 'cita noticia', '·w', 
                   'cita publicación', 'cite web', 'wayback', '!', 'esd', 'nowrap',
                   'cita requerida', 'cita vídeo', 'cita tuit', 'citarequerida', 
                   'cite tweet', 'bandera', 'bandera2', 'abbr', 'abreviatura', 
                   'rom', 'cr', 'per', 'sinref','url', 'estatura = [[1,67]]', 'nihongo',
                   'harvnb', 'harvnp', 'pagename', 'rtl-lang', 'hebreo', 'lang',
                   'tubo 8 hermanos i hermanas cuala identidad de algunos desconocemos uno murió en un acidente de coche',]
        
        
        ## ... VALUES
        
        if header in values_l:
            text = text.replace(tag, t_elements[1])
        
        
        ## ... MATRIMONIO
        
        elif header in matrimonio_l:         
            año_inicio = extract_rango_fechas(t_elements[2])[0]
            if len(año_inicio)>0:
                if len(t_elements)>3:
                    año_fin =  extract_rango_fechas(t_elements[3])[0]
                    if len(año_fin)>0:
                        replacement = f'[[{t_elements[1]}]] ({año_inicio}-{año_fin})'
                    else:
                        replacement = f'[[{t_elements[1]}]] ({año_inicio})'  
                else:
                    replacement = f'[[{t_elements[1]}]] ({año_inicio})'
            else:
                año_inicio = extract_rango_fechas(t_elements[3])[0]
                if len(año_inicio)>0:
                    if len(t_elements)>4:
                        año_fin =  extract_rango_fechas(t_elements[4])[0]
                        if len(año_fin)>0:
                            replacement = f'[[{t_elements[2]}]] ({año_inicio}-{año_fin})'
                        else:
                            replacement = f'[[{t_elements[2]}]] ({año_inicio})'
                    else:
                        replacement = f'[[{t_elements[2]}]] ({año_inicio})'
                        
                else:
                    replacement = ''
            
            
            divorcios_l = ['razón=div.', 'razón=anul.', 'razón=sep.', 'razón= div',
                           'fin=div', 'seacon=divorced', 'fin=div.', 'fin=divorciado',
                           'razón= div.', 'fin=fall.']
            fallecimientos_l = ['razón=fall.', 'fin=fall', 'fin=viu.', 'razón= su muerte']
            parejas_l = ['casado=no']
            
            femeninos_l = ['fem=sí']
            
            if any([e in t_elements for e in divorcios_l]):
                replacement = replacement + ' ==DIVORCIO=='
            elif any([e in t_elements for e in fallecimientos_l]):
                replacement = replacement + ' ==FALLECIMIENTO=='
            elif any([e in t_elements for e in parejas_l]):
                replacement = replacement + ' ==PAREJA==' 
                
            if any([e in t_elements for e in femeninos_l]):                         
                replacement = replacement

            text = text.replace(tag, replacement)
        
        
        ## ... FECHA
        
        elif header in fecha_l:
            number = re.findall(r'\d*', t_elements[1])
            if len(number)==0:
                replacement = ''
            elif len(extract_rango_fechas(t_elements[1])[0])>0:
                año = extract_rango_fechas(t_elements[1])[0]
                replacement = año
            elif len(t_elements)>6:
                replacement = f'(=<=FECHAS=<= {t_elements[1].strip()}/{t_elements[2].strip()}/{t_elements[3].strip()}-{t_elements[4].strip()}/{t_elements[5].strip()}/{t_elements[6].strip()} =>=FECHAS=>=)'
            else:
                try:
                    replacement = f'(=<=FECHA=<= {t_elements[1].strip()}/{t_elements[2].strip()}/{t_elements[3].strip()} =>=FECHA=>=)'
                except:
                    replacement = ''
                
            text = text.replace(tag, replacement)           
        
        
        ## ... LISTAS   
        
        elif header in flatlist_l:
            
            replacement = ' '.join(t_elements[1:])
            text = text.replace(tag, replacement)
            
        
        ## ... DESCARTAR
        
        elif header in descartar_l:
            text = text.replace(tag, ' ')
            
        elif len(t_elements)==1:
            text = text.replace(tag, t_elements[0])
            
        else:
            #print('****** !!!!!!! Brackets ... ******')
            #print('****** !!!!!!! ******')
            #print(t_elements)
            #print('****** !!!!!!! Brackets ... ******')
            #print('****** !!!!!!! ******')
            
            if len(t_elements)==2:
                text = text.replace(tag, t_elements[1])
            else:
                text = text.replace(tag, ' ')
    
    return text

####



def replace_char_out_of_brackets(text: str, char_to_replace, replacement_char=','):
    
    ''' This '''
    
    brackets_pattern = r'\[\[.*?\]\]'
    brackets = re.findall(brackets_pattern, text)
    b_replace = '<<$$$$$>>'
    for b in brackets:
        text = text.replace(b, b_replace)
       
    parentheses_pattern = r'\(.*?\)'
    parentheses = re.findall(parentheses_pattern, text)
    p_replace = '<<&&&&&>>'
    for p in parentheses:    
        text = text.replace(p, p_replace)
        
    text = text.replace(char_to_replace, replacement_char)
    
    s_l = text.split('<<&&&&&>>')
    text = ''
    for ind,s in enumerate(s_l[:-1]):
        text += (s + parentheses[ind])        
    text += s_l[-1]

    s_l = text.split('<<$$$$$>>')
    text = ''
    for ind,s in enumerate(s_l[:-1]):
        text += (s + brackets[ind])
    text += s_l[-1]
    
    return text

####


def remove_reference_tags(text: str):
    
    ''' ... '''
    
    pattern = r'<ref.*?</ref>'
    refs = re.findall(pattern, text)
    for ref in refs:
        text = ','.join(text.split(ref))
    
    pattern = r'<ref.*?/>'
    refs = re.findall(pattern, text)
    for ref in refs:
        text = ','.join(text.split(ref))
    
    return text

####


def detect_separators(text: str, references: list, brackets: list):
    
    ''' ... '''
    
    text = clean_text_I(text)
    
    text = decode_references(text, references)
    text = decode_brackets(text, brackets)
    
    text = decode_references(text, references)
    text = decode_brackets(text, brackets)
    
    text = remove_links(text)

    text = remove_reference_tags(text)    
    text = clean_text_I(text)
    
    separators = ['<br/>', '<br />', '<br/ >', '<br>', '<Br>', '·']
    for s in separators:
        text = text.replace(s, ',')
    
    separators = ['y [[']
    for s in separators:
        text = text.replace(s, ', [[')
    
    separators = [') [[']
    for s in separators:
        text = text.replace(s, '), [[')
        
    separators = [']] [[']
    for s in separators:
        text = text.replace(s, ']], [[')
    
    chars = ['-', '/', ';']
    for char in chars:
        text = replace_char_out_of_brackets(text, char)
    
    pattern = r',\s*?(J|j)r.*'
    text = re.sub(pattern, '&&&&&', text)
    
    text = replace_char_out_of_brackets(text, ',', '%%%%%')
    text = replace_info_from_brackets(text)
    
    text = text.replace('&&&&&',', Jr.')
    
    elements = text.split('%%%%%')
    elements = [clean_text_II(e) for e in elements if len(e.strip())>0]
    
    for e in elements:
        if len(e)>0:
            if e[0]=='*':
                if len(e)==1:
                    elements.remove(e)
                else:
                    e = e[1:].strip()
            else:
                e = e.strip()
        else:
            elements.remove(e)

    return elements

# Atributos

In [ ]:
print_attr=False

## Nombre artístico

In [ ]:
nombre_artistico_l = ['nombre', 'nombre artistico', 'nombres', 'naombre', 
                      'nombre de artistico', 'name', 'tambien llamado', 'conocida como',
                      'conocido como', 'popularmente conocida', 'seudonimo artistico',
                      'sobrenombre artistico', 'nombre en español',
                      'alias', 'apodo', 'seudonimo', 'apodos', 'nombre alias', 'aliaz',
                      'pseudonimos', 'pseudonimo',  'aias',
                      'alias(es)',  'mote', 'otros apodos',
                      'podo', 'conocido con', 'esposa:\nalias',
                      'otros nombres', 'nombre alternativo', 'otros nombre',
                      'nombre tnt',]

for key in nombre_artistico_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                e, date = extract_date(e)
                e, fecha = decode_fecha(e)
                e, fecha_1, fecha_2 = decode_fechas(e)
                e = remove_rango_fechas(e)
                e = remove_age(e)
                e = clean_text_II(e)
                
                if e!='':  

                    try:
                        a['nombre_']
                    except:
                        a['nombre_'] = dict()
                    try:
                        a['nombre_']['artistico']
                    except:
                        a['nombre_']['artistico'] = list()
                        
                    if print_attr:    
                        print(f'+ {e}')
                    a['nombre_']['artistico'].append(e)
                    
            if print_attr:
                print('****')
            del a[key]

## Nombre nacimiento

In [ ]:
nombre_nacimiento_l = ['nombre nativo', 'nombre de nacimientol', 'nombredenacimiento',
    '%«% 0 %»%|nombre de nacimiento', 'nombre de nacimient', 'nombre de nacimiento',
    'nombre de nacimiemto', 'nombrere de nacimiento', '[|nombre de nacimiento',
    'nombre de nacimiendo', 'nombre de macimiento', 'native name', 'birth name',
    'nombre nacimiento', 'mombre de nacimiento', 'nombre chino', 'nombre oficial',
    'nombre verdadero', 'nombre real', 'mombre real','nombre_de_nacimiento',
    'birth_name', 'nombre_real', 'nombre completo', 'nombre de completo',
    'nombrecompleto',]

for key in nombre_nacimiento_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                e, date = extract_date(e)
                e, fecha = decode_fecha(e)
                e, fecha_1, fecha_2 = decode_fechas(e)
                e = remove_rango_fechas(e)
                e = remove_age(e)
                e = clean_text_II(e)
                
                if e!='':  

                    try:
                        a['nombre_']
                    except:
                        a['nombre_'] = dict()       
                    try:
                        a['nombre_']['nacimiento']
                    except:
                        a['nombre_']['nacimiento'] = list()
                        
                    if print_attr:    
                        print(f'+ {e}')
                    a['nombre_']['nacimiento'].append(e)
                    
            if print_attr:
                print('****')
            del a[key]

## Ocupacion

In [ ]:
ocupacion_l = ['ocupación', '|ocupación', 'ocupacion', 'occupation', 'ocupacion(es)', 'ocupacion actual',
    'ocupacion/es', 'ocu5465416561435461352pacion', 'ocupacionn', 'ocupacion (es)',
    'ocupacoon', 'profesion', 'occupacion', 'ocupacionpacion', '[[ocupacion]]',
    'especialidades',]

for key in ocupacion_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                e, date = extract_date(e)
                e, fecha = decode_fecha(e)
                e, fecha_1, fecha_2 = decode_fechas(e)
                e = remove_rango_fechas(e)
                e = remove_age(e)
                e = clean_text_II(e)
                
                if e!='':  

                    try:
                        a['ocupacion_']
                    except:
                        a['ocupacion_'] = list()
                        
                    if print_attr:
                        print(f'+ {e}')
                    a['ocupacion_'].append(e)
                    
            if print_attr:
                print('****')
            del a[key]

## Nacimiento

In [ ]:
nacimiento_lugar_l = ['lugar de nacimiendo', 'lugas de nacimiento',
    'lugar de nacimiento', 'lugar nacimiento', 'lugar  de nacimiento',
    'pais de nacimiento', 'birth place', 'pais de origen',  'pais',
    'origen', 'origin',  'origen:', 'paisdenacimiento', 'nacido', 'nacimient',
    'fecha nacimiento', 'fecha de nacimiento', 'birth date', 'edad',
    'fecha de nacimiento por primera vez', 'nacio', 'nacimiento', 'born']

for key in nacimiento_lugar_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                e, date = extract_date(e)
                e, fecha = decode_fecha(e)
                e, fecha_1, fecha_2 = decode_fechas(e)
                e = remove_rango_fechas(e)
                e = remove_age(e)
                e = clean_text_II(e)
                
                # Lugar
                
                if e!='':    
                    
                    try:
                        a['nacimiento_']
                    except:
                        a['nacimiento_'] = dict() 
                    try:
                        a['nacimiento_']['lugar']
                    except:
                        a['nacimiento_']['lugar'] = list()
                    if print_attr:
                        print(f'lugar nacimiento: {e}')
                    a['nacimiento_']['lugar'].append(e)
                
                
                # Fechas
                
                fecha_nacimiento, fecha_muerte = get_born_death_dates([fecha, date, fecha_1, fecha_2],  b_d='b')
                
                if fecha_nacimiento != '':
                    
                    try:
                        a['nacimiento_']
                    except:
                        a['nacimiento_'] = dict() 
                    if print_attr:
                        print(f'fecha nacimiento: {fecha_nacimiento}')
                    a['nacimiento_']['fecha'] = fecha_nacimiento

                if fecha_muerte !='':
                    
                    try:
                        a['fallecimiento_']
                    except:
                        a['fallecimiento_'] = dict()
                    if print_attr:
                        print(f'fecha fallecimiento: {fecha_muerte}')
                    a['fallecimiento_']['fecha'] = fecha_muerte
            if print_attr:     
                print('****')
            del a[key]


## Nacionalidad

In [ ]:
nacionalidad_l = ['nacionaldiad','nacioanalidad', 'nacinalidad', 'nacionalidad (es)',
 'nacionalidad', 'ciudadania', 'ciudadania:']
 
for key in nacionalidad_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
            
                e, date = extract_date(e)
                e, fecha = decode_fecha(e)
                e, fecha_1, fecha_2 = decode_fechas(e)
                e = remove_rango_fechas(e)
                e = remove_age(e)
                e = clean_text_II(e)
                
                if e!='':
                    
                    try:
                        a['nacimiento_']
                    except:
                        a['nacimiento_'] = dict()
                    try:
                        a['nacimiento_']['nacionalidad']
                    except:
                        a['nacimiento_']['nacionalidad'] = list() 
                    
                    if print_attr:
                        print(f'nacionalidad: {e}')
                    a['nacimiento_']['nacionalidad'].append(e)
                    
            if print_attr: 
                print('****')
            del a[key]

## Fallecimiento

In [ ]:
fallecimiento_l = ['fecha de defuncion', 'lugar de defuncion', 'death place',
       'lugar de muerte', 'fallece', 'death date', 'fallecimento', 'muerte',
       'defuncion', 'lugar muerte', 'fecha de muerte', 'morte', 'fallecimiento',
       'death date', 'defuncion', 'fecha de fallecimiento',
       'lugar de fallecimiento',]

for key in fallecimiento_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                e, date = extract_date(e)
                e, fecha = decode_fecha(e)
                e, fecha_1, fecha_2 = decode_fechas(e)
                e = remove_rango_fechas(e)
                e = remove_age(e)
                e = clean_text_II(e)
                
                
                # Lugar
                
                if e!='': 
                    
                    try:
                        a['fallecimiento_']
                    except:
                        a['fallecimiento_'] = dict() 
                    try:
                        a['fallecimiento_']['lugar']
                    except:
                        a['fallecimiento_']['lugar'] = list()
                    
                    if print_attr:
                        print(f'lugar fallecimiento: {e}')
                    a['fallecimiento_']['lugar'].append(e)
                
                
                # Fechas
                
                fecha_nacimiento, fecha_muerte = get_born_death_dates([fecha, date, fecha_1, fecha_2])
                
                if fecha_muerte != '':
                    
                    try:
                        a['fallecimiento_']
                    except:
                        a['fallecimiento_'] = dict() 
                    
                    if print_attr:
                        print(f'fecha fallecimiento: {fecha_muerte}')
                    a['fallecimiento_']['fecha'] = fecha_muerte


                if fecha_nacimiento !='':
                    
                    try:
                        a['nacimiento_']
                    except:
                        a['nacimiento_'] = dict()
                    
                    if print_attr:
                        print(f'fecha nacimiento: {fecha_nacimiento}')
                    a['nacimiento_']['fecha'] = fecha_nacimiento
                 
            if print_attr:
                print('****')
            del a[key]

## Fallecimiento causa

In [ ]:
fallecimiento_causa_l = ['causa muerte', 'causa de muerte', 'causa de la muerte',
    'causa  de fallecimiento',]

for key in fallecimiento_causa_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                if e!='':
                    
                    try:
                        a['fallecimiento_']
                    except:
                        a['fallecimiento_'] = dict()

                    if print_attr:
                        print(f'+ {e}')
                    a['fallecimiento_']['causa'] = e
            
            if print_attr:
                print('****')
            del a[key]

## Periodo actividad

In [ ]:
años_actividad_l = ['desde', 'años activo de vida', 'años acivo', 'añoso activo',
    'años activo', 'years active', 'año debut', 'periodo de actividad', 'actividad',
    'años activa', 'años de actividad', 'tiempo activo', 'añoactivo', 'periodo actual',
    'año activo', 'periodo de\n actividad', 'añoa activeo', 'años activos',
    'año de debut', 'periodo actividad', 'en actividad', 'años activo de actividad',
    'anni di attivita', 'años en actividad', 'años en activo', 'activo', 'debut',
    'añoa activo', 'tiempo', 'periodo', 'trayectoria',]

for key in años_actividad_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['periodo_actividad_']
            except:
                a['periodo_actividad_'] = list()

            elements = ['-'.join(elements)]
            for e in elements:
                
                year_start, year_end = extract_rango_fechas(e)
                
                if (year_start!='') | (year_end!=''):
                    
                    periodo = {
                         'año_inicio': year_start,
                         'año_final': year_end,
                    }
                    
                    if print_attr:
                        print(periodo)
                    a['periodo_actividad_'].append(periodo)
            
            if print_attr:
                print('****')
            del a[key]

## Residencia

In [ ]:
residencia_l = ['radicada', 'residencia', 'vive en', 'ubicacion', 'residenciado',
    'localizacion', 'lugar de residencia', 'reside', 'ciudad de residencia',
    'domicilio', 'lugar', 'ciudad',]
for key in residencia_l:
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])

            for e in elements:
                
                e, date = extract_date(e)
                e, fecha = decode_fecha(e)
                e, fecha_1, fecha_2 = decode_fechas(e)
                e = remove_rango_fechas(e)
                e = remove_age(e)
                e = clean_text_II(e)
                
                if e!='':
                    
                    try:
                        a['residencia_']
                    except:
                        a['residencia_'] = list()
                
                    if print_attr:
                        print(f'+ {e}')
                    a['residencia_'].append(e)
            
            if print_attr:
                print('****')
            del a[key]

## Pareja

In [ ]:
pareja_l = ['\'\'\'pareja\'\'\'', 'prometida', 'novio', 'pareja', 'pareja.',
    'novia', 'novia actual', 'parejas', 'pareja actual', 'ocupacion    \npareja',]

for key in pareja_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
                
            for e in elements:
                
                person = remove_rango_fechas(e)
                person, reason = decode_reasons(person)
                relation = 'pareja'
                year_start, year_end = extract_rango_fechas(e)
                reason = ''

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_final': year_end,
                     'razón_fin': reason,
                }
                
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
            
            if print_attr:
                print('****')        
            del a[key]

## Conyugue

In [ ]:
conyuge_l = ['cónyuge', 'spouse', 'conjugue', 'conyuge', 'conyugue', 'esposa', 'conguye', 'conyuges',
    'casado con', 'nombre de la esposa', 'conyuge(s)', 'conyugues', 'conjuges',
    'conyuje', 'concuye', 'esposo', ] 

for key in conyuge_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])

            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
                
            for e in elements:
                
                person = remove_rango_fechas(e)
                person, reason = decode_reasons(person)
                relation = 'conyugue'
                year_start, year_end = extract_rango_fechas(e)
                
                if reason=='pareja':
                    reason=''
                    relation='pareja'

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_final': year_end,
                     'razón_fin': reason,
                }
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
                
            if print_attr:
                print('****')
            del a[key]

## Divorciados

In [ ]:
divorciados_l = ['divorciado con',]


for key in divorciados_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
                
            for e in elements:  
                
                person = remove_rango_fechas(e)
                relation = 'conyugue'
                year_start, year_end = extract_rango_fechas(e)
                reason = 'divorcio'

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_final': year_end,         # corregir
                     'razón_fin': reason,
                }
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
                
            if print_attr:
                print('****')
            del a[key]

## Padre

In [ ]:
padre_l = ['papa', 'padre', 'nombre del padre',]

for key in padre_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])

            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
                
            for e in elements:
                
                person = remove_rango_fechas(e)
                relation = 'progenitor'
                year_start, year_end = extract_rango_fechas(e)
                reason = ''

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_fin': year_end,
                     'razón_fin': reason,
                }
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
                            
            if print_attr:
                print('****')
            del a[key]

## Hijos

In [ ]:
hijo_l = ['hijos', 'hijas', 'hijo/s', 'hijos.', 'hijos(as)', 'hijo(a)s',
          'hijo', 'hijo(s)', 'children',]

for key in hijo_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])

            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
                
            for e in elements:           
                person = remove_rango_fechas(e)
                person, reason = decode_reasons(person)
                relation = 'hijo'
                year_start, year_end = extract_rango_fechas(e)
                
                try:
                    num_hijos = int(person)
                    
                except:

                    relacion = {
                         'persona': person,
                         'relación': relation,
                         'año_inicio': year_start,
                         'año_fin': year_end,
                         'razón_fin': reason,
                    }
                    if print_attr:
                        print(relacion)
                    a['relaciones_'].append(relacion)
                
            if print_attr:
                print('****')
            del a[key]

## Padre

In [ ]:
parientes_l = ['padres',]

for key in parientes_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])

            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
            
            for e in elements:
                
                person = remove_rango_fechas(e)
                relation = 'progenitor'
                year_start, year_end = extract_rango_fechas(e)
                reason = ''

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_fin': year_end,
                     'razón_fin': reason,
                }
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
                
            if print_attr:
                print('****')
            del a[key]

## Madre

In [ ]:
madre_l = ['madre', 'nombre de la madre',]

for key in madre_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
            
            for e in elements:
                
                person = remove_rango_fechas(e)
                relation = 'progenitor'
                year_start, year_end = extract_rango_fechas(e)
                reason = ''

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_fin': year_end,
                     'razón_fin': reason,
                }
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
                
            if print_attr:
                print('****')
            del a[key]

## Hermano

In [ ]:
hermano_l = ['hermanos', 'hermano', 'hermanas', 'hermana',]

for key in hermano_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
                
            for e in elements:
                
                person = remove_rango_fechas(e)
                relation = 'hermano'
                year_start, year_end = extract_rango_fechas(e)
                reason = ''

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_fin': year_end,
                     'razón_fin': reason,
                }
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
                
            if print_attr:
                print('****')
            del a[key]

## Expareja

In [ ]:
expareja_l = ['expareja',]

for key in expareja_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
                
            for e in elements:
                
                person = remove_rango_fechas(e)
                relation = 'pareja'
                year_start, year_end = extract_rango_fechas(e)
                reason = ''

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_fin': year_end,
                     'razón_fin': reason ,
                }
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
                
            
            if print_attr:
                print('****')
            del a[key]

## Maestro

In [ ]:
maestro_l = ['maestros', 'alumno de', 'alumna de']

for key in maestro_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
                
            for e in elements:
                
                person = remove_rango_fechas(e)
                relation = 'maestro'
                year_start, year_end = extract_rango_fechas(e)
                reason = ''

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_fin': year_end,
                     'razón_fin': reason ,
                }
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
                
            
            if print_attr:
                print('****')
            del a[key]

## Alumno

In [ ]:
alumno_l = ['alumnos']

for key in alumno_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['relaciones_']
            except:
                a['relaciones_'] = list()
            
            for e in elements:
                
                person = remove_rango_fechas(e)
                relation = 'alumno'
                year_start, year_end = extract_rango_fechas(e)
                reason = ''

                relacion = {
                     'persona': person,
                     'relación': relation,
                     'año_inicio': year_start,
                     'año_fin': year_end,
                     'razón_fin': reason ,
                }
                if print_attr:
                    print(relacion)
                a['relaciones_'].append(relacion)
                
            
            if print_attr:
                print('****')
            del a[key]

## Estudios

In [ ]:
estudios_l = ['alma mater', 'almamater', 'posgrado', 'estudios',]

for key in estudios_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
                
            for e in elements:
                e = remove_rango_fechas(e)
                
                if e!='':
                    try:
                        a['estudios_']
                    except:
                        a['estudios_'] = list()
                
                    if print_attr:
                        print(f'+ {e}')
                    a['estudios_'].append(e)
                
            if print_attr:
                print('****')
            del a[key]

## Grupos

In [ ]:
grupos_l = ['relacionados', 'artistas relacionados', 'relacionado/a con',
    'relacionada con', 'cantantes relacionados', 'aristas relacionados', 'relacionado',
    'artistas relaconados', 'relacionadoa', 'artistas asociados', 'relacionados:',
    'artistas\n relacionados', 'artistas ralacionados', 'colaboraciones',
    'influencias', 'influido por', 'influences', 'influenciado por',
    'agrupacion', 'miembro de', 'grupo', 'grupos', 'banda', 'bandas',
    'bandas asociadas', 'grupos asociados', 'grupos relacionados',
    'bandas anteriores',]

for key in grupos_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                if e!='':
                    
                    try:
                        a['grupos_']
                    except:
                        a['grupos_'] = list()
                        
                    if print_attr:
                        print(f'+ {e}')
                    a['grupos_'].append(e)
                
            if print_attr:
                print('****')
            del a[key]

## Instrumentos

In [ ]:
instrumento_l = ['instrumento   ,', 'istrumento', 'instrumento (s)', 
    'instrumento preferido', 'instrumento )', 'instrumento(s)',
    'instruments', 'instrumento', 'instrumentos', 'instrument', 'intrumentos',]

for key in instrumento_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                if e!='':

                    try:
                        a['instrumentos_']
                    except:
                        a['instrumentos_'] = dict()   
                    try:
                        a['instrumentos_']['tipo']
                    except:
                        a['instrumentos_']['tipo'] = list()
                    
                    if print_attr:
                        print(f'+ {e}')
                    a['instrumentos_']['tipo'].append(e)
            
            if print_attr:
                print('****')
            del a[key]

## Instrumentos modelos

In [ ]:
instrumento_especifico_l = ['notable instruments', 'intrumentos notables',
    'instrumento preferido(s)', 'instrumentos notables', 'instrumentos musicales',
    'instrumento destacado', 'instrumentos destacados', 'instrumento notable',
    'equipo musical', 'guitarras',]

for key in instrumento_especifico_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                if e!='':

                    try:
                        a['instrumentos_']
                    except:
                        a['instrumentos_'] = dict()     
                    try:
                        a['instrumentos_']['modelo']
                    except:
                        a['instrumentos_']['modelo'] = list()
                    
                    if print_attr:
                        print(f'+ {e}')
                    a['instrumentos_']['modelo'].append(e)
            
            if print_attr:
                print('****')
            del a[key]


## Voz

In [ ]:
voz_l = ['tesitura', 'voz', 'voz.', 'tipo de voz', 'registro vocal', 'tipo de voz|voz',
    'tipo de [[voz (musica)%%voz]]', 'categoria de voz', 'tipo de [[voz]]',
    'tipo de voz:', 'registro de voz', 'tipo de voz.', 'tipo_de_voz', ]


for key in voz_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                if e!='':
                    
                    if print_attr:
                        print(f'+ {e}')
                    a['tipo_voz_'] = e
            
            if print_attr:
                print('****')
            del a[key]

## Géneros

In [ ]:
generos_l = ['género', 'alias|estilo', 'genergo', 'genero (s)', 'genero musical', 'genero',
    'estilo', 'genre', 'generos', 'tipo de musica', 'estilyo', 'estilo musical',
    'genero(s)',]

for key in generos_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                if e!='':

                    try:
                        a['generos_']
                    except:
                        a['generos_'] = list()
                    
                    if print_attr:
                        print(f'+ {e}')
                    a['generos_'].append(e)
            
            if print_attr:
                print('****')
            del a[key]

## Discográficas

In [ ]:
discografica_l = ['discográfica', 'compañía discográfica', 'compañia discografica url',
    'compañia discografical', 'compañia', 'etiquetas', 'label',
    'compañia discografica', 'discografica', 'compañia discograficas',
    'compañia discografica(s)', 'discograficas', 'discogafica',
    'sello', 'discrografica', 'discografico(s)', 'compañia discografica actual',
    'compañia discografica (s)', 'discogrfica', 'compañia discografica/s',
    'sellos', 'discografica(s)', 'compañias discograficas', 'sello discografico',
    'discografica &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;', 'siscografica',]

for key in discografica_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            try:
                a['discograficas_']
            except:
                a['discograficas_'] = list()
                
            for e in elements:
                
                year_start, year_end = extract_rango_fechas(e)
                discography = remove_rango_fechas(e)

                discografica = {
                     'discografica': discography,
                     'año_inicio': year_start,
                     'año_fin': year_end
                }
                if print_attr:
                    print(discografica)
                a['discograficas_'].append(discografica)
            
            if print_attr:
                print('****')
            del a[key]

## Discografía

In [ ]:
discografia_l = [ 'discografía', 'discos', 'albumes', 'discografia', 'albums', 'discographia',
    'estudio de grabacion','albumexitoso', 'album mas exitoso', 'album exitoso',]

for key in discografia_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:  
                disc = remove_rango_fechas(e)
                year = extract_rango_fechas(e)[0]
                
                if disc!='':
                    try:
                        a['discografia_']
                    except:
                        a['discografia_'] = list()
                
                    d = {'disco': disc,
                         'año': year}
                    
                    if print_attr:
                        print(d)
                    a['discografia_'].append(d)
                
            if print_attr:
                print('****')
            del a[key]

## Single

In [ ]:
single_l = ['sencillo más exitoso', 'single mas exitoso', 'sencilloexitoso',
    'sencillo mas exitoso', 'sencillo exitoso', 'cancion debut', 'sencillo', 
    'mejor cancion',]

for key in single_l:
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
        
            try:
                a['singles_']
            except:
                a['singles_'] = list()
                
            for e in elements:
                
                song = remove_rango_fechas(e)
                if len(elements)==1:
                    year = extract_rango_fechas(e)[0]
                else:
                    year = extract_rango_fechas(song)[0]

                single = {'cancion': song,
                          'año': year
                }
                
                if print_attr:
                    print(single)
                a['singles_'].append(single)
                
            if print_attr:
                print('****')
            del a[key]

## Idioma

In [ ]:
idioma_l = ['lengua artistica', 'idiomas', 'idioma', 'lengua materna',]

for key in idioma_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])
            
            for e in elements:
                
                if e!='':

                    try:
                        a['idioma_']
                    except:
                        a['idioma_'] = list()
                    
                    if print_attr:
                        print(f'+ {e}')
                    a['idioma_'].append(e)
            
            if print_attr:
                print('****')
            del a[key]

## Religones

In [ ]:
religion_l = ['religion', 'religion.', 'fe', 'creencias religiosas',
              'creencia religiosa', 'fé',]
for key in religion_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])

            try:
                a['religiones_']
            except:
                a['religiones_'] = list()
                
            for e in elements:
                
                e, date = extract_date(e)
                e, fecha = decode_fecha(e)
                e, fecha_1, fecha_2 = decode_fechas(e)
                e = remove_rango_fechas(e)
                e = remove_age(e)
                e = clean_text_II(e)
                
                if print_attr:
                    print(f'+ {e}')
                a['religiones_'].append(e)
                
            if print_attr:
                print('****')
            del a[key]

## Premios

In [ ]:
premios_otros_l = ['premios', 'otros premios', 'premio(s)', 'distinciones',]

for key in premios_otros_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])

            for e in elements:           
                prize = remove_rango_fechas(e)
                prize = clean_premios(prize)
                year_start, yeras_end = extract_rango_fechas(e)

                if (year_start!='') | (prize!=''):
                    
                    try:
                        a['premios_']
                    except:
                        a['premios_'] = list()
                    
                    premio = {
                         'tipo': prize,
                         'año': year_start,
                    }
                    if print_attr:
                        print(premio)
                    a['premios_'].append(premio)
                
            if print_attr:
                print('****')
            del a[key]

## Otros premios

In [ ]:
premios_2_l = ['premios cesar', 'premios grammy', 'premios oscar', 'premios sag',
               'premios bafta', 'premios ariel', 'premios globo de oro',
               'premios goya', 'premios condor',]

for key in premios_2_l:
    print('\n\n............................')
    print(key)
    print('............................')
    for a in data_persons:
        keys_l = list(a.keys())
        if key in keys_l:
            if print_attr:
                print(a[key])
            
            elements = detect_separators(a[key],
                                         a['_metadata_html_'],
                                         a['_metadata_brackets_'])

            for e in elements:   
                
                prize = remove_rango_fechas(e)
                prize = clean_premios(prize)
                prize = key + ' - ' + prize
                prize = clean_text_II(prize)
                
                year_start, yeras_end = extract_rango_fechas(e)

                if (year_start!='') | (prize!=''):
                    
                    try:
                        a['premios_']
                    except:
                        a['premios_'] = list()
                
                    premio = {
                         'tipo': prize, 
                         'año': year_start,
                    }
                    if print_attr:
                        print(premio)
                    a['premios_'].append(premio)
                
            if print_attr:
                print('****')
            del a[key]

_____

In [ ]:
print(f'# of articles about persons: {len(data_persons)}')
attr_frequency_persons = get_attr_frequency(data_persons)

In [ ]:
attr_frequency_persons

______

## Text

In [ ]:
def clean_info_from_brackets(text: str):
    
    ''' ... '''
    
    brackets_pattern = '\[\[.*?\]\]'
    brackets = re.findall(brackets_pattern, text)
    
    for bracket in brackets:
        parts = bracket.split('%%')
        if len(parts)==1:
            replace_text = bracket.replace('[','').replace(']','')     
        else:
            replace_text = parts[1].replace(']','')
        
        text = text.replace(bracket, replace_text)

    return text

####

def remove_brackets(text: str):

    ''' ... '''
        
    pattern = '%{%.*?%}%'
    tags = re.findall(pattern, text)
    
    for tag in tags:       
        text = text.replace(tag, ' ')
 
    return text

####

def remove_urls(text: str):
    
    text = re.sub(r'''\[?(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]\]?))''', "", text)

    return text

####

def remove_references(text: str):

    ''' ... '''

    text = re.sub(r'<.*?>','',text)

    pattern = '%«%.*?%»%'
    tags = re.findall(pattern, text)
    
    for tag in tags:       
        text = text.replace(tag, ' ')
 
    return text

####

def replace_note_separator(text: str):
    
    left_index = 0
    right_index = len(text)

    while True:
        reg_resp_l = re.search(r'\[\[', text[left_index:right_index])

        if reg_resp_l == None:
            break

        else:

            left_index = reg_resp_l.span()[0] + left_index
            reg_resp_r = re.search(r'\]\]', text[left_index:right_index])

            if reg_resp_r == None:
                print('¡warning!')
                print('-------')
                return text

            else:

                right_index = reg_resp_r.span()[1] + left_index
                note = text[left_index:right_index]
                note_replacement = note.replace('|', '%%')

                text = note_replacement.join(text.split(note))

                left_index = right_index
                right_index = len(text) 
    
    return text

In [ ]:
texto = ['text']

for key in texto:
    print('\n\n............................')
    print(key)
    print('............................')
    for ind,a in enumerate(data_persons):
        keys_l = list(a.keys())
        if key in keys_l:
            text = a[key]
            
            splits = text.split('\n')
            
            text_II = ''
            first = True
            second = True
            for i,s in enumerate(splits):
                if first:
                    if len(s)>40:
                        text = s
                        first = False
                        continue
                else:
                    if second:
                        text_II = s
                        second = False
                    if len(s)>40:
                        text_II = s
                        break
                
            texts = [text, text_II]
            
            for ind,t in enumerate(texts):
                t = remove_urls(t)
                t = remove_brackets(t)
                t = remove_reference_tags(t)
                t = remove_references(t)
                t = replace_note_separator(t)
                t = t.strip()
                t = clean_info_from_brackets(t)
                t = clean_text_II(t, texto=True)
                texts[ind] = t
            
            a['text'] = texts[0]
            a['text_II'] = texts[1]

## Save data processed

In [ ]:
pickle.dump(data_persons, open( "../Data/data_v3_persons.p", "wb"))

_____